In [2]:
from biolab import *
from datetime import datetime
from pathlib import Path
from pymongo import ASCENDING

DB = Path("~/datasets/").expanduser()
DB_BASE_ = DB / Path("iris_db/")
DS_ID = "IITD_v1"
DS_NAME_ = Path(DS_ID)
DS_BASE_ = DB_BASE_ / DS_NAME_
from lib import FileManager as FM
from lib import LoggerManager as LM
lg = LM.get_logger(name=__name__, level="INFO")
fm = FM()

fm.ensure_exists(DS_BASE_)


In [3]:
# BASE_ = DB / "iris_datasets/"
ORIG_DB_BASE_ = Path("~/alam/iris_normalization/segmented_iitd_bmp").expanduser()


In [5]:
new_img_tag = "seg_irisseg"
"""
Iris segmentated images after applying from mask from IRISSEG-EP dataset
"""
new_img_tag_info = {
    'ds_id': DS_ID,
    new_img_tag: {
        "info": "Iris segmentated images after applying from mask from IRISSEG-EP dataset",
        "img_specs": {
            "ext": ".bmp",
            "width": 320,
            "height": 240
        },
        "doc": "https://docs.google.com/document/d/1N_ninklrJYcEyG0EUuSx_KmBeHvnOYrtBTEw1yMZXNM/edit?tab=t.i0h2chnldv7h",
        "old_base_": str(ORIG_DB_BASE_.as_posix()),
        "git": "https://github.com/nishkalprakash/alam_iris_normalize/blob/master/iitd_iris_extration.ipynb"
    }
}


In [6]:

with IrisMeta() as meta:
    # meta.insert_new_img_tag(new_img_tag, new_img_tag_info)
    print(meta.update(new_img_tag_info))
    print(meta.coll.update_one(
        {'ds_id': DS_ID},
        {'$unset': {f"{new_img_tag}.orig_base_path": ""}}
    ))
    print(meta.list_ds())




Found matches for meta: meta
UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)
UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)
{'IITD_v1', 'CASIA_iris_thousand', 'CASIA_v1'}


In [4]:
# images = sorted(list(ORIG_DB_BASE_.rglob(f"*{new_img_tag_info[new_img_tag]['img_specs']['ext']}")))
# print(len(images))

In [5]:
# # from PIL import Image
# ds_id = DS_ID
# db = IrisDB(ds_id=DS_ID)
# import re
# from tqdm import tqdm
# docs=[]
# for img_ in tqdm(images):
#     ext = img_.suffix
#     # name = OperatorA_001-A_02
#     #             
#     # pattern = r'^(\d+)_[LR]_(\d+)$'  # Regex pattern to match the filename structure
#     # match = re.match(pattern, img_.stem)
#     # if not match:
#         # lg.warning(f"Filename {img_.name} does not match the expected pattern.")
#         # continue
#     # person_id, eye, sample_id = map(str,map(int, match.groups()))
#     image_id = img_.stem
#     doc = db.find_one({'image_id': image_id},{
#         'image_id': 1,
#         'eye_id': 1,
#         'img_tags':1, 
#     })
#     new_filename_ = Path(f"{image_id}{ext}")
#     path_ = DS_BASE_ / new_img_tag/ doc['eye_id'] / new_filename_
#     # img=Image.open(img_)
#     # w,h = img.size
#     # img_specs = {
#         # 'ext': ext,
#         # 'width': w,
#         # 'height': h,
#     # }
#     doc[new_img_tag] = {
#         'orig_rel_path': str(img_.relative_to(ORIG_DB_BASE_)),
#     }
#     # if new_img_tag not in doc['img_tags']:
#     try:
#         if new_img_tag not in doc['img_tags']:
#             doc['img_tags'].append(new_img_tag)
#     except KeyError:
#         doc['img_tags'] = ['orig',new_img_tag]

#     db.update(doc)
#     fm.copy_file(img_, path_)

# # db.insert(docs)
# db.close()

In [6]:
with IrisDB(ds_id=DS_ID) as db:
    print(db.get_stats(new_img_tag))

Found matches for IITD_v1: IITD_v1
{'num_images': 2240, 'num_people': 224, 'num_eyes': 435, 'num_eyes_per_person_count': {'1': 13, '2': 211}, 'num_samples_per_eye_count': {'5': 416, '10': 13, '6': 3, '4': 3}, 'num_sessions': 1}


In [11]:
# db.get_tag_data('orig')

with IrisMeta() as meta:
    for ds_id in meta.list_ds():
        with IrisDB(ds_id=ds_id) as db:
            for tag in meta.find_one({'ds_id': ds_id})['img_tags']:
                stats= db.get_stats(tag)
                # append to {tag}.stats
                print(meta.coll.update_one({'ds_id': ds_id}, {'$set': {f'{tag}.stats': stats}}))

Found matches for meta: meta
Found matches for IITD_v1: IITD_v1
UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)
UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)
UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)
UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)
Found matches for CASIA_v1: CASIA_v1
UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)
Found matches for CASIA_iris_thousand: CASIA_iris_thousand
UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)
